---
layout: post
title: Trimester 2 Final Exam Retrospective
categories: [AP CSA]
menu: nav/CSA_Units/final-exam-retrospective.html
permalink: /csa/final-exam-retrospective.html
---

# Outline

### 9/10 Points
1. 5 points - **5 Things I Did**: Issues, Burn Down, lesson(s), presentation(s), analytics, personal blog
2. 2 points - **Full Stack Feature Demo**: highlighting Java and N@tM feedback
3. 1 point - **Visualization** (drawIO, mermaid) in your blog
4. 1 point - **MCQ / FRQ Work**

### 10th Point
5. **Retrospective** interest in another N@TM project, looking ahead, individual strenghts + weaknesses, self grade

# **5 Things I Did**

| **Task** | **Evidence/Links** | **Description** |
|----------|--------------------|----------------|
| **Admin Approval Request** | [Issue](https://github.com/orgs/CSA-Coders-2025/projects/3/views/10?pane=issue&itemId=95881868&issue=CSA-Coders-2025%7CPlanning-Repository-Issue-House-%7C111) <br> ![Image](https://github.com/user-attachments/assets/bc048c8d-8304-4f1e-b453-7172796acad1) | Developed a page for the admin to approve/deny student requests for queue entry. Implemented frontend request handling, backend UI status updates, and approval/denial logic to modify the database. |
| **FRQ Team Teach on Array/ArrayList** | ![Image](https://github.com/user-attachments/assets/557b8552-4260-4ed0-a96c-ac8c5d10cf11) <br> ![Image](https://github.com/user-attachments/assets/b1999ec7-8729-475c-8cb9-83ef0b0e7f0f) | Led a session covering arrays and ArrayLists, including review exercises, interactive "popcorn" coding challenges, scoring strategies, and application to official scoring criteria. Also assigned homework for reinforcement. |
| **Bathroom Queue & Hall Pass Management** | ![Image](https://github.com/user-attachments/assets/d98a9da2-6b0d-495f-afa1-6039a08c69b9) | Enhanced the bathroom pass and hall pass system to incorporate approval requests. Implemented retrieval of `timeIn` upon approval to track student movement. |
| **Admin Alert System** | ![Image](https://github.com/user-attachments/assets/d839bed7-3ab6-4750-8897-f36ddbd0c3eb) <br> ![Image](https://github.com/user-attachments/assets/5e3112a1-1e60-49d0-957e-1d57c5de9b69) | Created an alert system that notifies the admin of pending student approvals directly from the navigation page. |
| **(Additional) Student Panel** | [Issue](https://github.com/LiliWuu/lilianw_2025/issues/6) <br> [Blog](https://github.com/LiliWuu/lilianw_2025/blob/b4dca399952e4aa9a08e50d4ae8a428cbadf57c5/_notebooks/Foundation/csa/2024-01-11-student-panel.ipynb) | Attended a student panel to gain insights into CS and engineering students' experiences in college. Learned key takeaways from Mr. M’s courses and received advice on essential skills for success in STEM fields. |


# Full Stack Feature Demo
> Updated Feature Flowchart

[Link](https://lucid.app/lucidchart/7cb3bd66-b04f-4558-a944-941918afc069/edit?viewport_loc=-987%2C-206%2C2860%2C3004%2C0_0&invitationId=inv_7ec41f05-3434-489f-9078-be7ba2f91266)

<img width="573" alt="Image" src="https://github.com/user-attachments/assets/05612f21-aba9-4349-b8c5-25bca5ce9f0f" />

## Frontend
### HTTP Request for Approval
Sends a request to the approval request endpoint rather than the addToQueue endpoint to await approval before being added to the queue. 

![Image](https://github.com/user-attachments/assets/b731ace0-314a-48a2-a6f4-d1573db6453d)

> Bathroom.ipynb

```java
    window.addToQueue = function() {
        const requestData = {
            teacherEmail: teacherName,
            studentName: window.studentName,
        };

        fetch(approvalURI + "/sendApprovalRequest", {
            method: "POST",
            headers: { "Content-Type": "application/json" },
            body: JSON.stringify(requestData),
        })
        .then(response => {
            if (response.ok) {
                alert("Approval request sent. Please wait for the teacher’s decision.");
                console.log(`Approval request sent for: ${window.studentName}`);
            } else {
                alert("Failed to request approval.");
            }
        })
        .catch(error => console.error("Error requesting approval:", error));
    };
```

### Retrieve timeIn from Backend
Method to get the timeIn from backend through a fetch request

> Hallpass.ipynb

```java
    async function getTimeIn(studentName) {
        const response = await fetch(timeInUrl(studentName));

        if (response.ok) {
            const tinkleData = await response.text(); // Ensure text response
            console.log(`Retrieved timeIn: ${tinkleData}`);
            return tinkleData;
        } else {
            console.warn("Could not retrieve timeIn for", studentName);
            return null;
        }
    }

    window.approveStudent = async function() {
        const timeIn = await getTimeIn(window.studentName);

        /** rest of code */
    }  

```

## Backend
### Approval Request DB
Created a new DB for students awaiting approval. Only after approval are they added to the bathroom queue DB. 

```java
    @PostMapping("/sendApprovalRequest")
        public ResponseEntity<Object> sendApprovalRequest(@RequestBody ApprovalRequest requestDto) {
            ApprovalRequest newRequest = new ApprovalRequest(requestDto.getTeacherEmail(), requestDto.getStudentName(), null);
            approvalRepository.save(newRequest);
        
            return new ResponseEntity<>("Approval request sent successfully!", HttpStatus.CREATED);
    }
```

<img width="1034" alt="Image" src="https://github.com/user-attachments/assets/53ebac52-4c10-4132-baa1-7eaa96dc524d" />

### Admin Approval Dashboard
A thymeleaf page that displays students awaiting approval in a table format and allows the admin to approve/deny. Record and store timeIn upon approval.

```java
// Approve Student: Remove from requests & Add to Queue
        function approveStudent(teacherEmail, studentName) {
            const now = new Date();
            const hours = now.getHours().toString().padStart(2, "0");
            const minutes = now.getMinutes().toString().padStart(2, "0");
            const seconds = now.getSeconds().toString().padStart(2, "0");
            const timeIn = `${hours}:${minutes}:${seconds}`;

            fetch(javaURI + "/api/approval/approveRequest", {
                method: "POST",
                headers: { "Content-Type": "application/json" },
                body: JSON.stringify({ teacherEmail, studentName, timeIn }) 
            }).then(response => {
                if (response.ok) {
                    console.log(`Approved: ${studentName}, now adding to queue with timeIn: ${timeIn}`);
                    fetchPendingRequests(); // Refresh table after approval
                } else {
                    console.log(`Failed to approve request.`);
                }
            });
        }

        // Deny Student: Remove from requests only
        function denyStudent(teacherEmail, studentName) {
            fetch(javaURI + "/api/approval/denyRequest", { 
                method: "DELETE",
                headers: { "Content-Type": "application/json" },
                body: JSON.stringify({ teacherEmail, studentName })
            }).then(response => {
                if (response.ok) {
                    console.log(`Denied: ${studentName}`);
                    fetchPendingRequests(); // Refresh table after denial
                } else {
                    alert("Failed to deny request.");
                }
            });
        }

```

### Retrieve and Store timeIn
Get timeIn from requestDto, then parse and format it. Store the timeIn in a hash map to be used later.

> ApprovalRequestApiController.java

```java
 @PostMapping("/approveRequest")
    public ResponseEntity<Object> approveRequest(@RequestBody ApprovalRequest requestDto) {
    String timeInRaw = requestDto.getTimeIn();
        if (timeInRaw == null || timeInRaw.trim().isEmpty()) {
            System.out.println("ERROR: Received empty timeIn value");
            return new ResponseEntity<>("Invalid time format", HttpStatus.BAD_REQUEST);
        }

        LocalTime parsedTimeIn = LocalTime.parse(timeInRaw);
        String formattedTimeIn = parsedTimeIn.format(DateTimeFormatter.ofPattern("HH:mm:ss")); // 24-hour format

        timeInMap.put(student.getName(), formattedTimeIn);

        /** rest of code */
    }
```

> TinkleApiController.java
Retrieves timeIn from hash map stored in memory and can be accessed by frontend. 

```java
 @GetMapping("/timeIn/{studentName}")
    public ResponseEntity<Object> getTimeIn(@PathVariable String studentName) {
        System.out.println("🔍 Fetching timeIn for: " + studentName);
    
        // Retrieve stored timeIn from memory (ApprovalRequestApiController)
        String timeIn = ApprovalRequestApiController.getTimeInFromMemory(studentName);

        if (timeIn != null) {
            System.out.println("Retrieved timeIn from memory for " + studentName + ": " + timeIn);
            return ResponseEntity.ok(timeIn); // Return timeIn value
        } else {
            System.out.println("Student not found in memory: " + studentName);
            return ResponseEntity.status(HttpStatus.NOT_FOUND).body("Student not found");
        }
    }
```



# MCQ & FRQ

### First Practice Exam
<img width="1381" alt="Image" src="https://github.com/user-attachments/assets/40c269d5-b78a-4700-97ed-3e5bcdb7d12f" />

### Second Practice Exam
<img width="1372" alt="Image" src="https://github.com/user-attachments/assets/37a0eb5d-54e6-4947-84ac-a8b54af91af2" />

[2014 Practice FRQ](https://github.com/LiliWuu/lilianw_2025/blob/6b0c7bb97406143e909fdf349f41fc099ce646dd/_notebooks/Foundation/csa/Review/2024-03-02-2014-frq.ipynb)


---

# N@TM Feedback
[Google Form](https://docs.google.com/forms/d/1i0TwNtcie104jUWGtAy2gJKNf996-i294QpqYoqLx-c/edit#responses)

### Informative Feedback
**"I think one major thing you could implement is some method of ensuring that people are actually going to the bathroom."**
- This is a good point but implementing it may be difficult.

**"Leaderboard for students who use the bathroom the most, weekly and monthly stats."**
- Could allow for integration with other groups working on leaderboard. Also could encourage better bathroom behavior.

**"Include MFA such as a key code to prevent spoofing another student's bathroom pass."**
- Could be a concern for scanner since theres no MFA required.

# 🌟 Retrospective
## Exploring Other Projects at N@TM 
![Image](https://github.com/user-attachments/assets/05e1912c-34bb-4cfb-b926-911355c44ca6)
![Image](https://github.com/user-attachments/assets/d1f93832-2280-448d-9a78-07082103c40d)
- Learned about the Blackjack game
- I thought it was good that they integrated their feature with stocks so that a player's money from stocks is translated into Blackjack
- I liked that they had leaderboard with 
*Suggestions...*
- Adding a multiplayer option using web sockets 
- They could add different game modes (competition mode, challenge mode)
- Integrate learning into the game (having players answer coding questions to proceed to the next game)

## Looking Ahead
- Learning about AI/machine learning concepts and applications in CS class
- Summer internships/programs based on applications of coding (COSMOS AI for Speech and Imagery, REHS Developing Jupyter Notebooks to Run On the ICICLE Project)
- Career in CS/engineering


## Individual Strengths & Weaknesses

| **Strengths**  | **Weaknesses** | 
|-------|-----|
| Detail-oriented | Poor presentation, speaking, commmuncation skills  | 
| Collaboration   | Not communicating effectively  | 
| Quick learner   | Not speaking out more  |
| Hard worker   | Lack of skill/knowledge  | 


# Self-Grade

| **Points Available**  | **Topic** | **My Points** | **Reasoning** | 
|-------|-----|-----|-----|
| 5 | 5 Things I Did  | 5 |I captured the major things I did over Tri 2 and had evidence of blogs, issues, images, code for the 5 things I did.|
| 2   | Full Stack Feature Demo  | 2 | I included code snippets, visualizations, and explanations of different parts of my feature. |
| 1   | Visualization  | 1  | Redid my flowchart to be more improved and contain my most updated features.|
| 1   | MCQ / FRQ Work  | 1  | Did the second practice collegeboard MCQ and did significantly better. I went over the problems I missed.|
| 1   | Retrospective  | 1 | My retrospective included demonstrated my interest in another project at N@TM as well as a thoughtful review of their project. I included a section on how CS would look like in my future and what I plan to do. I also included a self-grade and individual strengths and weaknesses.|
| **Total Points:** 10  | | **My Total Points** 10| 


